#### LangChain

从本质上分析，LangChain还是采用从大模型自身出发的策略，通过开发人员在实践过程中对打模型能力的深入理解及其在不同场景下的涌现能力，使用模块化的方式进行高级抽象，设计出统一接口以适配各种大模型。到目前为止，LangChain抽象出最终的核心模块如下：

1. Model I/O：标准化各个大模型的输入和输出，包含输入模板，模型本身和格式化输出；
2. Retrieval：检索外部数据，然后在执行生成步骤时将其传递到LLM，包括文档加载、切割、Embedding等；
3. Chains：链条，LangChain框架中最重要的模块，链接多个模块协同构建应用，是实际运作很多功能的高级抽象；
4. Memory：记忆模块。以各种方式构建历史信息，维护有关实体及其关系的信息；
5. Agents：目前最热门的Agents开发事件，未来能够真正实现通用人工智能的落地方案；
6. Callbacks：回调系统，允许连接到LLM应用程序的各个阶段。用于日志记录、监控、流传输和其他任务

### 1. 在LangChain中使用在线智谱大模型

In [6]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from dotenv import load_dotenv
from rich import print
import os
load_dotenv()
api_key = os.getenv("ZHIPU_API_KEY")

print(api_key)

c65b3dee6b47dc164cc804c4f9426f37.bKUcNHj2Tarnkdid

In [7]:
# 初始化智谱AI聊天模型
chat = ChatZhipuAI(
    model="glm-4",
    api_key=api_key,
    temperature=0.7,
    )

In [8]:
# 构建消息
messages = [
    AIMessage(content="Hi."),
    SystemMessage(content="你是一名诗人"),
    HumanMessage(content="请你帮我写一首关于鸟的诗")
]

In [ ]:
response = chat.invoke(messages)
print(response.content)

### 2. 什么是ChatPromptTemplate 

ChatPromptTemplate是LangChain提供的一个提示词模板工具，专门用于构建对话类模型的输入。

这些提示模板用于格式化消息列表

In [23]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "你是一位乐于助人的小助理"),
    ("user", "什么是{topic}")
])
prompt_template.invoke({"topic":"什么是快乐星球"})
# chat.invoke(prompt_template.invoke({"topic":"什么是快乐星球"})).content

ChatPromptValue(messages=[SystemMessage(content='你是一位乐于助人的小助理', additional_kwargs={}, response_metadata={}), HumanMessage(content='什么是什么是快乐星球', additional_kwargs={}, response_metadata={})])

### 3. 如何构建Chat Models的LLMChain链路

In [30]:
# from langchain.chains import LLMChain
# chain = LLMChain(prompt=prompt_template, llm=chat)
# chain.invoke({"topic":"什么是快乐星球"})["text"]

# 推荐使用LCEL的语法格式构建链路
chain = prompt_template | chat  
chain.invoke({"topic":"什么是快乐星球"}).content

'快乐星球是一部中国大陆的儿童科幻电视剧，首播于2004年。这部电视剧讲述了一个来自快乐星球的神秘小机器人多啦A梦，为了拯救地球，与地球上的孩子们一起经历各种冒险的故事。\n\n在剧中，快乐星球是一个充满科技和和谐氛围的星球，那里的居民拥有高度发达的科技和丰富的想象力。快乐星球的小机器人多啦A梦来到地球，帮助地球上的孩子们解决各种问题，传播正能量，同时也让小朋友们了解到科技的力量和友谊的重要性。\n\n快乐星球因其独特的科幻元素、寓教于乐的内容以及富有童趣的角色设计，受到了广大小朋友的喜爱，成为了中国儿童电视剧的经典之作。'

### 4. Callbacks

Callbacks(回调函数)是一种编程模式,Callbacks的意义在于:允许程序在某个任务完成时自动执行另一个函数,而不必阻塞等待某个长时间运行的操作完成.所以它会在处理异步操作,如网络请求文件读写或任何可能需要等待的操作时频繁被使用

对于与大模型的连续交互场景, 一般采用的都是流式输出. 但在做流式输出前, 需要的就是回调Callbacks

回调处理程序可以是sync(同步)或async(异步)的. 同步回调和异步回调是编程中常用的两种处理事件或数据的方式,它们在执行时和用途上有明显的区别:

1. 同步回调(Synchronous Callbacks): 
    - 执行时机: 同步回调实在主程序流程中直接调用和执行的. 当一个同步回调函数被触发时, 程序会立即执行这个回调函数, 并且在回调函数执行完成之前, 主程序流程会被阻塞
    - 用途: 同步回调通常用于确保某些操作必须在程序继续执行前完成. 例如, 在访问数据中的每个元素并对齐应用函数时, 你可能会使用数据的`.foreach()`方法, 这是一个同步的回调实现
2. 异步回调(Asynchronous Callbacks):
    - **执行时机**: 异步回调不会立即执行,它们通常被放置在事件队列中,等待当前执行堆栈清空后才开始执行. 这意味着程序的主流程不会等待异步回调的完成, 可以继续执行其他任务
    - **用途**: 异步回调通常用在不希望阻塞主程序流程的情况下, 例如处理I/O操作(如网络请求, 文件读写)

在并发编程的Web开发中, 交互式应用对话场景,肯定采用的是异步的回调

In [ ]:
from typing import Any, Dict, List
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate
from rich import print
from dotenv import load_dotenv
from langchain.chains import LLMChain
import os
load_dotenv()
api_key = os.getenv("ZHIPU_API_KEY")

class LoggingHandler(BaseCallbackHandler):
    def on_chat_model_start(
            self, serialized, messages, *, run_id, parent_run_id = None, tags = None, metadata = None, **kwargs
            ) -> None:
        # 可选：打印模型侧启动信息
        print("Chat model started")

    def on_llm_end(
            self, response, *, run_id, parent_run_id = None, **kwargs
            ):
        print(f"Chat model ended, response: {response}")
    
    def on_chain_start(
            self, serialized, inputs, *, run_id, parent_run_id = None, tags = None, metadata = None, **kwargs
            ):
        # 安全获取可读名称：优先用传入的 run_name，其次尝试从 serialized 中提取，最后兜底类型名/Unknown
        run_name = kwargs.get("run_name")
        name = None
        if isinstance(serialized, dict):
            name = serialized.get("name") or serialized.get("id") or serialized.get("lc") or serialized.get("type")
        if not name:
            name = run_name or ("None" if serialized is None else type(serialized).__name__)
        inputs_info = list(inputs.keys()) if isinstance(inputs, dict) else type(inputs).__name__
        print(f"Chain {name} started; inputs: {inputs_info}, {run_name}")

    def on_chain_end(
            self, outputs, *, run_id, parent_run_id = None, **kwargs
            ):
        print(f"Chain ended, outputs: {outputs}")

# 定义回调
callbacks = [LoggingHandler()]

# 实例化语言模型
llm = ChatZhipuAI(model="glm-4", api_key=api_key)

# 定义提示模板
prompt = ChatPromptTemplate.from_template("1 + {num}等于多少")

chain = (prompt | llm).with_config(run_name="AddChain")
# 这里保留 LLMChain 并在调用时传入 run_name
# chain = LLMChain(prompt=prompt, llm=llm)

response = chain.invoke({"num": 2}, config={"callbacks": callbacks, "run_name": "AddChain"})
#  

In [ ]:
print(response.content)

#### 用回调机制流式地调用GLM-4

首先下载httpx_sse,pip install httpx_sse

http_sse是一个基于httpx地Server-Sent Events(SSE)客户端库。SSE是一张服务器推送技术，允许服务器通过HTTP链接向客户端发送推送消息。这种技术常用于实时消息传递和通知，如股票价格更新、新闻更新或其他需要从服务器到客户端实时通信的应用场景

In [ ]:
from langchain_community.chat_models import ChatZhipuAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.manager import CallbackManager
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token, *, chunk = None, run_id, parent_run_id = None, **kwargs):
        print(f"MY cunstom handler, token: {token}")

prompt = ChatPromptTemplate.from_messages(["请给我将一个关于{animal}的笑话"])

llm = ChatZhipuAI(
    api_key=api_key, 
    model="glm-4",
    streaming=True,
    # callbacks=[MyCustomHandler()] # 自定义流式输出
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),   # 标准流式输出
    )

chain = prompt | llm
response = chain.invoke({"animal": "熊"})
response

### 5. Memory

Memory作为存储记忆数据的一个抽象模块，其作为一个独立模块使用是没有任何意义的，因为本质上它的定位就是一个存储对话数据的空间。先抛开其内部实现的复杂性，我们可以回想一下：在定义链路的时候，每个链的内部都会根据其接收到的输入去定义其核心执行逻辑，比如在链内如何去调用外部工具，如何解析返回的数据格式等。其中链路收到的输入，可以直接来自用户，同时，也可以来自Memory模块。所以在这个过程中，一个链如果接入了Memory模块，其内部会与Memory模块进行两次交互：



1. 收到用户输入之后，执行核心逻辑之前，链会读取Memory模块，拿到对应的数据，与用户输入的Prompt放在一起，执行接下来的逻辑。
2. 执行核心逻辑之后，返回响应之前，链会将这个过程中产生的信息，写入Memory模块，以便在其他场景下能够引用到这些记忆数据。

由此可见，Memory模块Chains模块是一对相互协作的关系，就像我们定义Prompt一样，不同的需求们需要构建不同的Prompt Template。那我们想记录下哪些数据，不论是用户输入的prompt，还是大模型的相应结果，亦或者是链路的中间过程生成的数据，全部就由Memory这个抽象模块来完成。所以这个模块最核心干的就是两件事：如何存储数据和如何提取出句，对应着的就是两个基本操作：读和写。

In [18]:
from langchain_community.chat_models import ChatZhipuAI

chat = ChatZhipuAI(
    api_key=api_key,
    model="glm-4",
    temperature=0.7
)

In [19]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("你好")
memory.chat_memory.add_ai_message("你好，请问有什么可以帮你的?")

C:\Users\kk\AppData\Local\Temp\ipykernel_8616\648014646.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


In [20]:
memory.load_memory_variables({})

{'history': 'Human: 你好\nAI: 你好，请问有什么可以帮你的?'}

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("你好")
memory.chat_memory.add_ai_message("你好，请问有什么可以帮你的?")

In [22]:
memory.load_memory_variables({})

{'chat_history': 'Human: 你好\nAI: 你好，请问有什么可以帮你的?'}

In [24]:
from langchain_community.chat_models import ChatZhipuAI

from langchain_core.prompts import(
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatZhipuAI(
    api_key=api_key,
    model="glm-4",
    temperature=0.7
)
prompt = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template("你是一个具有上下文记忆能力的对话机器人"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [25]:
from langchain.chains import LLMChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [26]:
conversation.invoke({"question":"你好"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个具有上下文记忆能力的对话机器人
Human: 你好

> Finished chain.


{'question': '你好',
 'chat_history': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好👋！有什么可以帮助你的吗？', additional_kwargs={}, response_metadata={})],
 'text': '你好👋！有什么可以帮助你的吗？'}

In [27]:
conversation.invoke({"question":"今天天气怎么样"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个具有上下文记忆能力的对话机器人
Human: 你好
AI: 你好👋！有什么可以帮助你的吗？
Human: 今天天气怎么样

> Finished chain.


{'question': '今天天气怎么样',
 'chat_history': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好👋！有什么可以帮助你的吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='今天天气怎么样', additional_kwargs={}, response_metadata={}),
  AIMessage(content='很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？', additional_kwargs={}, response_metadata={})],
 'text': '很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？'}

In [28]:
conversation.invoke({"question":"现在适合出去玩儿吗？"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个具有上下文记忆能力的对话机器人
Human: 你好
AI: 你好👋！有什么可以帮助你的吗？
Human: 今天天气怎么样
AI: 很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？
Human: 现在适合出去玩儿吗？

> Finished chain.


{'question': '现在适合出去玩儿吗？',
 'chat_history': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好👋！有什么可以帮助你的吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='今天天气怎么样', additional_kwargs={}, response_metadata={}),
  AIMessage(content='很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='现在适合出去玩儿吗？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='要判断现在是否适合出去玩儿，需要考虑以下几个因素：\n\n1. **天气情况**：查看当前的气温、湿度、风力等是否适宜户外活动。\n2. **空气质量**：空气质量指数（AQI）是否良好，适合户外活动。\n3. **个人健康状况**：你是否感到舒适，没有不适感。\n4. **当地疫情情况**：根据当地疫情防控政策，是否允许外出。\n\n如果你能提供你所在地的天气和空气质量信息，我可以帮助你更好地判断是否适合外出。否则，你可以根据自己的感受和当地情况来决定。', additional_kwargs={}, response_metadata={})],
 'text': '要判断现在是否适合出去玩儿，需要考虑以下几个因素：\n\n1. **天气情况**：查看当前的气温、湿度、风力等是否适宜户外活动。\n2. **空气质量**：空气质量指数（AQI）是否良好，适合户外活动。\n3. **个人健康状况**：你是否感到舒适，没有不适感。\n4. **当地疫情情况**：根据当地疫情防控政策，是否允许外出。\n\n如果你能提供

In [29]:
conversation.invoke({"question":"我叫kk，希望你能记得我"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个具有上下文记忆能力的对话机器人
Human: 你好
AI: 你好👋！有什么可以帮助你的吗？
Human: 今天天气怎么样
AI: 很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？
Human: 现在适合出去玩儿吗？
AI: 要判断现在是否适合出去玩儿，需要考虑以下几个因素：

1. **天气情况**：查看当前的气温、湿度、风力等是否适宜户外活动。
2. **空气质量**：空气质量指数（AQI）是否良好，适合户外活动。
3. **个人健康状况**：你是否感到舒适，没有不适感。
4. **当地疫情情况**：根据当地疫情防控政策，是否允许外出。

如果你能提供你所在地的天气和空气质量信息，我可以帮助你更好地判断是否适合外出。否则，你可以根据自己的感受和当地情况来决定。
Human: 我叫kk，希望你能记得我

> Finished chain.


{'question': '我叫kk，希望你能记得我',
 'chat_history': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好👋！有什么可以帮助你的吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='今天天气怎么样', additional_kwargs={}, response_metadata={}),
  AIMessage(content='很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='现在适合出去玩儿吗？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='要判断现在是否适合出去玩儿，需要考虑以下几个因素：\n\n1. **天气情况**：查看当前的气温、湿度、风力等是否适宜户外活动。\n2. **空气质量**：空气质量指数（AQI）是否良好，适合户外活动。\n3. **个人健康状况**：你是否感到舒适，没有不适感。\n4. **当地疫情情况**：根据当地疫情防控政策，是否允许外出。\n\n如果你能提供你所在地的天气和空气质量信息，我可以帮助你更好地判断是否适合外出。否则，你可以根据自己的感受和当地情况来决定。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我叫kk，希望你能记得我', additional_kwargs={}, response_metadata={}),
  AIMessage(content='当然可以，kk！我已经记住你了。有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={})],
 '

In [30]:
conversation.invoke({"question":"真的吗，你真的记住我的名字了吗，我叫什么"})



> Entering new LLMChain chain...
Prompt after formatting:
System: 你是一个具有上下文记忆能力的对话机器人
Human: 你好
AI: 你好👋！有什么可以帮助你的吗？
Human: 今天天气怎么样
AI: 很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？
Human: 现在适合出去玩儿吗？
AI: 要判断现在是否适合出去玩儿，需要考虑以下几个因素：

1. **天气情况**：查看当前的气温、湿度、风力等是否适宜户外活动。
2. **空气质量**：空气质量指数（AQI）是否良好，适合户外活动。
3. **个人健康状况**：你是否感到舒适，没有不适感。
4. **当地疫情情况**：根据当地疫情防控政策，是否允许外出。

如果你能提供你所在地的天气和空气质量信息，我可以帮助你更好地判断是否适合外出。否则，你可以根据自己的感受和当地情况来决定。
Human: 我叫kk，希望你能记得我
AI: 当然可以，kk！我已经记住你了。有什么我可以帮助你的吗？
Human: 真的吗，你真的记住我的名字了吗，我叫什么

> Finished chain.


{'question': '真的吗，你真的记住我的名字了吗，我叫什么',
 'chat_history': [HumanMessage(content='你好', additional_kwargs={}, response_metadata={}),
  AIMessage(content='你好👋！有什么可以帮助你的吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='今天天气怎么样', additional_kwargs={}, response_metadata={}),
  AIMessage(content='很抱歉，由于我无法访问实时数据，我无法提供今天的具体天气情况。你可以查看当地的天气预报或者使用天气应用程序来获取最新的天气信息。需要我帮你查找如何获取这些信息的方法吗？', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='现在适合出去玩儿吗？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='要判断现在是否适合出去玩儿，需要考虑以下几个因素：\n\n1. **天气情况**：查看当前的气温、湿度、风力等是否适宜户外活动。\n2. **空气质量**：空气质量指数（AQI）是否良好，适合户外活动。\n3. **个人健康状况**：你是否感到舒适，没有不适感。\n4. **当地疫情情况**：根据当地疫情防控政策，是否允许外出。\n\n如果你能提供你所在地的天气和空气质量信息，我可以帮助你更好地判断是否适合外出。否则，你可以根据自己的感受和当地情况来决定。', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我叫kk，希望你能记得我', additional_kwargs={}, response_metadata={}),
  AIMessage(content='当然可以，kk！我已经记住你了。有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata=